In [3]:
import pandas as pd

# Veri setinin URL üzerinden okunması
url = "https://raw.githubusercontent.com/guipsamora/pandas_exercises/master/07_Visualization/Online_Retail/Online_Retail.csv"

# Verinin okunması
df = pd.read_csv(url, encoding="ISO-8859-1")

# İlk 5 satıra bakılır
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/10 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/10 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/10 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/10 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/10 8:26,3.39,17850.0,United Kingdom


In [4]:
# 1. Veri seti hakkında genel bilgi alınır
print("--- Temizlik Öncesi Bilgiler ---")
df.info()

--- Temizlik Öncesi Bilgiler ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


In [5]:
# 2. Hangi sütunda kaç tane Null değeri olduğuna bakılır
print("\n--- Eksik Veri Sayıları ---")
print(df.isnull().sum())


--- Eksik Veri Sayıları ---
InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64


In [6]:
# 3. CustomerID'si olmayan satırları silinir
df.dropna(subset=['CustomerID'], inplace=True)

# 4. Temizlik sonrası kontrol edilir
print("\n--- Temizlik Sonrası Satır Sayısı ---")
print(df.shape)


--- Temizlik Sonrası Satır Sayısı ---
(406829, 8)


In [7]:
# 1. Önce negatif değer var mı diye istatistiksel özetine bakılır
print("---Temizlik Öncesi İstatistikler---")
print(df.describe().T)

---Temizlik Öncesi İstatistikler---
               count          mean          std      min       25%       50%  \
Quantity    406829.0     12.061303   248.693370 -80995.0      2.00      5.00   
UnitPrice   406829.0      3.460471    69.315162      0.0      1.25      1.95   
CustomerID  406829.0  15287.690570  1713.600303  12346.0  13953.00  15152.00   

                 75%      max  
Quantity       12.00  80995.0  
UnitPrice       3.75  38970.0  
CustomerID  16791.00  18287.0  


In [8]:
# 2. Fatura numarası 'C' (Cancel) ile başlayanlar filtrelenir
df=df[~df["InvoiceNo"].str.contains("C", na=False)]

In [9]:
# 3. Miktarı (Quantity) 0'dan büyük olanları alınır
df=df[df["Quantity"]>0]

In [10]:
# 4. Son durumu kontrol edilir
print("\n--- İadeler Temizlendikten Sonra Satır Sayısı ---")
print(df.shape)


--- İadeler Temizlendikten Sonra Satır Sayısı ---
(397924, 8)


In [11]:
# tarih ve saat hesaplamaları yapmamızı sağlayan kütüphane çağırılır
import datetime as dt

In [12]:
# 1. Toplam Tutar (Ciro) Hesabı
df["TotalPrice"]= df["Quantity"]*df["UnitPrice"]

In [13]:
# 2. Tarih Formatını Düzeltme (String -> Datetime)
df["InvoiceDate"]= pd.to_datetime(df["InvoiceDate"])

/tmp/ipython-input-1971810879.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["InvoiceDate"]= pd.to_datetime(df["InvoiceDate"])


In [14]:
# 3. Analiz Tarihini (Bugünü) Belirleme
today_date= df["InvoiceDate"].max()+dt.timedelta(days=2)

In [15]:
print(f"Belirlediğimiz Analiz Tarihi: {today_date}")
df.head()

Belirlediğimiz Analiz Tarihi: 2011-12-11 12:50:00


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalPrice
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34


In [17]:
# Müşteri Kimliğine (CustomerID) göre gruplanması
rfm= df.groupby('CustomerID').agg({
    'InvoiceDate': lambda date: (today_date - date.max()).days,
    'InvoiceNo': lambda num: num.nunique(),
    'TotalPrice': lambda price: price.sum()
    })

In [18]:
rfm.columns= ['Recency', 'Frequency', 'Monetary']   #sütun isimlerini düzenledik
rfm= rfm[rfm["Monetary"]>0] # recency(bugün alışveriş yapan) sayısı 0 olanları matematikte sorun çıkmaması için 1 yaparız
print(rfm.head())

            Recency  Frequency  Monetary
CustomerID                              
12346.0         327          1  77183.60
12347.0           3          7   4310.00
12348.0          76          4   1797.24
12349.0          20          1   1757.55
12350.0         311          1    334.40


In [19]:
# 1. Recency Skoru (Ters Mantık: Küçük değer en yüksek puanı alır)
rfm["RecencyScore"] = pd.qcut(rfm['Recency'], 5, labels=[5, 4, 3, 2, 1])

In [20]:
# 2. Frequency Skoru (Düz Mantık: Büyük değer en yüksek puanı alır)
rfm["FrequencyScore"] = pd.qcut(rfm['Frequency'].rank(method="first"), 5, labels=[1, 2, 3, 4, 5])

In [24]:
# 3. Monetary Skoru (Düz Mantık: Büyük değer en yüksek puanı alır)
rfm["MonetaryScore"] = pd.qcut(rfm['Monetary'], 5, labels=[1, 2, 3, 4, 5])

In [22]:
# 4. Skorların Birleştirilmesi (String İşlemi)
rfm["RFM_SCORE"] = (rfm['RecencyScore'].astype(str) +
                    rfm['FrequencyScore'].astype(str))

In [23]:
print(rfm.head())

            Recency  Frequency  Monetary RecencyScore FrequencyScore  \
CustomerID                                                             
12346.0         327          1  77183.60            1              1   
12347.0           3          7   4310.00            5              5   
12348.0          76          4   1797.24            2              4   
12349.0          20          1   1757.55            4              1   
12350.0         311          1    334.40            1              1   

           MonetaryScore RFM_SCORE  
CustomerID                          
12346.0                5        11  
12347.0                5        55  
12348.0                4        24  
12349.0                4        41  
12350.0                2        11  


In [26]:
# 1. Segment Haritası (Regex - Regular Expressions)
seg_map = {
    r'[1-2][1-2]': 'Hibernating',
    r'[1-2][3-4]': 'At_Risk',
    r'[1-2]5': 'Cant_Loose',
    r'3[1-2]': 'About_to_Sleep',
    r'33': 'Need_Attention',
    r'[3-4][4-5]': 'Loyal_Customers',
    r'41': 'Promising',
    r'51': 'New_Customers',
    r'[4-5][2-3]': 'Potential_Loyalists',
    r'5[4-5]': 'Champions'
}

In [27]:
# 2. replace fonksiyonu ile skorlar isimlere çevrilir
rfm['Segment'] = rfm['RFM_SCORE'].replace(seg_map, regex=True)

In [28]:
# 3. Sonuç: Artık her müşterinin bir "Sınıfı" var
print(rfm[["Recency", "Frequency", "Monetary", "Segment"]].head())

            Recency  Frequency  Monetary      Segment
CustomerID                                           
12346.0         327          1  77183.60  Hibernating
12347.0           3          7   4310.00    Champions
12348.0          76          4   1797.24      At_Risk
12349.0          20          1   1757.55    Promising
12350.0         311          1    334.40  Hibernating


In [29]:
# Segmentlerin ortalamasını alarak grubun karakterine karar verilir.
segment_analizi = rfm[["Segment", "Recency", "Frequency", "Monetary"]].groupby("Segment").agg(["mean", "count"])

print(segment_analizi)

                        Recency        Frequency           Monetary      
                           mean count       mean count         mean count
Segment                                                                  
About_to_Sleep        54.504274   351   1.162393   351   461.061510   351
At_Risk              156.062069   580   2.865517   580  1076.506433   580
Cant_Loose           133.428571    63   8.380952    63  2796.155873    63
Champions              6.876777   633  12.417062   633  6857.935482   633
Hibernating          218.897653  1065   1.101408  1065   487.707579  1065
Loyal_Customers       34.469166   827   6.458283   827  2856.720328   827
Need_Attention        54.064516   186   2.327957   186   889.226398   186
New_Customers          7.857143    42   1.000000    42   388.212857    42
Potential_Loyalists   18.123984   492   2.010163   492  1034.905467   492
Promising             24.444444    99   1.000000    99   355.351313    99
